In [6]:
%config IPCompleter.greedy=True
from tensorflow.python.keras.preprocessing import image as pimage
from tensorflow.python.keras.applications.inception_v3 import *
from tensorflow.python.keras.layers import Flatten
from tensorflow.python.keras.models import Model
import numpy as np
import os

In [7]:
# def parse_data_inception(data):
inception = InceptionV3(include_top=False, weights='imagenet', 
                        input_tensor=None, input_shape=None, pooling="max")

preds = Flatten()(inception.output)
model = Model(inception.input, preds)

In [8]:
class1_path = '/home/nerses/Desktop/ml/3rd part/ml_homework/image_net/tiny-imagenet-200/train/n01774750/images'
class2_path = '/home/nerses/Desktop/ml/3rd part/ml_homework/image_net/tiny-imagenet-200/train/n01983481/images'


In [9]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, None, None, 3 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, None, None, 3 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [12]:
def parse(path_in, path_out):
    try:
        os.mkdir(path_out)
    except Exception as e:
        print(e)
    
    for image in os.listdir(path_in):
        file = os.path.join(path_in,image)
        img = pimage.load_img(file, target_size=(299, 299))
        x = pimage.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        y = model.predict(x).squeeze()
        np.save(os.path.join(path_out,image[:-5]), y)

parse(class1_path, 'c1')
parse(class2_path, 'c2')
    

In [85]:
from sklearn.cluster import KMeans, MiniBatchKMeans, AgglomerativeClustering, DBSCAN

In [14]:
data1 = np.array([np.load(os.path.join('c1',npy)) for npy in os.listdir('c1')])
data2 = np.array([np.load(os.path.join('c2',npy)) for npy in os.listdir('c2')])


In [19]:
data = np.concatenate([data1,data2],axis=0)
data.shape

(1000, 2048)

In [21]:
cluster = KMeans(n_clusters=2, random_state=0).fit(data)

In [29]:
# this is percentage of right guesses for each class

print(len(cluster.labels_[:500][cluster.labels_[:500]==0])/5)
print(len(cluster.labels_[500:][cluster.labels_[500:]==1])/5)

63.2
50.2


In [102]:
dbs = DBSCAN(eps=40, min_samples=3).fit(data)
np.unique(dbs.labels_, return_counts=True)


(array([-1,  0,  1]), array([958,  35,   7]))

In [104]:
# print(len(dbs.labels_[:500][dbs.labels_[:500]==1])/5)
# print(len(dbs.labels_[500:][dbs.labels_[500:]==0])/5)

print(len(dbs.labels_[:500][dbs.labels_[:500]==0])/5)
print(len(dbs.labels_[500:][dbs.labels_[500:]==1])/5)

7.0
1.4


In [78]:
import hdbscan
clusterer = hdbscan.HDBSCAN(min_cluster_size=4, gen_min_span_tree=True)
clusterer.fit(data)

HDBSCAN(algorithm='best', allow_single_cluster=False, alpha=1.0,
    approx_min_span_tree=True, cluster_selection_method='eom',
    core_dist_n_jobs=4, gen_min_span_tree=True, leaf_size=40,
    match_reference_implementation=False, memory=Memory(cachedir=None),
    metric='euclidean', min_cluster_size=4, min_samples=None, p=None,
    prediction_data=False)

In [103]:
# print(len(clusterer.labels_[:500][clusterer.labels_[:500]==1])/5)
# print(len(clusterer.labels_[500:][clusterer.labels_[500:]==0])/5)

print(len(clusterer.labels_[:500][clusterer.labels_[:500]==0])/5)
print(len(clusterer.labels_[500:][clusterer.labels_[500:]==1])/5)

17.2
4.8


In [46]:
agl = AgglomerativeClustering(n_clusters=2, linkage="ward").fit(data)

In [47]:

print(len(agl.labels_[:500][agl.labels_[:500]==1])/5)
print(len(agl.labels_[500:][agl.labels_[500:]==0])/5)

45.6
99.4


In [ ]:
# results are bad becacuse of choosing small dataset i guess, ill try the same for bigger one later
# dataset is tinyimagenet